In [ ]:
# import math
import random
from random import shuffle
import numpy as np

class Domino:
    
    def buy(self,hand, n, pieces):
        i = 0
        while i<n:
            hand.append(pieces[i])
            i=i+1
        del pieces[0:n]
        return hand

    #recebe estado do jogo, retorna ações possíveis
    def possibleActions(self,state,player):
        status = state[0]
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
        l_end = state[4]
        r_end = state[5]
        actions = []
        index = -1

        if(l_end==r_end==-1):
            for piece in hand:
                index +=1
                actions.append((piece[0],piece[1],index))
            return actions

        for piece in hand:
            index +=1
            #peça dupla
            if(piece[0]==piece[1]):
                if(piece[0]==l_end):
                    actions.append((piece[1],"left",index))
                elif(piece[1]==r_end):
                    actions.append((piece[0],"right",index))
                continue
            if(piece[0]==l_end):
                actions.append((piece[1],"left",index))
                if(l_end==r_end): #evitar duplicacao de mesmas ações na esq e dir
                    continue
            if(piece[1]==l_end):
                actions.append((piece[0],"left",index))
                if(l_end==r_end):
                    continue
            if(piece[0]==r_end):
                actions.append((piece[1],"right",index))
                if(l_end==r_end):
                    continue
            if(piece[1]==r_end):
                actions.append((piece[0],"right",index))
                if(l_end==r_end):
                    continue
        '''
        pos 2 e 3 no actions se referem a nova ponta na mesa:
        exemplo: campo 1-5 , mão : (1,2),(5,5)
        nesse caso a peça (1,2) se encaixa no lado esquerdo, sendo
        a nova ponta esquerda o 2. A representação será: (0,1), pois
        0 é a posição da tupla na mão, e o 1 representa a posição na tupla,
        que no caso é o direito

        '''
        if not actions:
            actions.append(None)
        return actions

    def termination(self,state):#se a partida ja terminou
        status = state[0]
        p1_hand = state[1]
        p2_hand = state[2]
        l_end = state[4]
        r_end = state[5]
        if (status >= 3):
            return True  
        return False

    def reward(self,state,player):#recompensa por estar nesse estado
        #em caso de vitoria
        status = state[0]

        if(status == 1 or status == 2):
            return 0
        if(status == 3):
            p1_hand = state[1]
            p2_hand = state[2]
            lighestP1Value = 13
            lighestP2Value = 13
            lighestP1Tile = 13
            lighestP2Tile = 13
            p1_total_value = 0
            p2_total_value = 0
            for piece in p1_hand:
                value = piece[0]+piece[1]
                p1_total_value+=value
            for piece in p2_hand:
                value = piece[0]+piece[1]
                p2_total_value+=value
            if(p1_total_value<p2_total_value):
                state[0] = 4#player 1 venceu tendo a menor mao
                if(player == "p1"):
                    return 1
                else:
                    return -1
            elif(p1_total_value>p2_total_value):
                state[0] = 5#player 2 venceu tendo a menor mao
                if(player == "p1"):
                    return -1
                else:
                    return 1
            else:#empate
                for piece in p1_hand:
                    if(piece[0]<lighestP1Value):
                        lighestP1Value = piece[0]
                        lighestP1Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP1Value):
                        lighestP1Value = piece[1]
                        lighestP1Tile = piece[0]+piece[1]
                        
                for piece in p2_hand:
                    if(piece[0]<lighestP2Value):
                        lighestP2Value = piece[0]
                        lighestP2Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP2Value):
                        lighestP2Value = piece[1]
                        lighestP2Tile = piece[0]+piece[1]
                        
                if(lighestP1Value<lighestP2Value):
                    state[0] = 4#player 1 venceu tendo o menor valor entre o par das pecas    
                    if(player == "p1"):
                        return 1
                    else:
                        return -1
                elif(lighestP1Value>lighestP2Value):
                    state[0] = 5#player 2 venceu tendo o menor valor entre o par das pecas
                    if(player == "p1"):
                        return -1
                    else:
                        return 1
                else:
                    if(lighestP1Tile<lighestP2Tile):#menor valor da peça como um todo
                        state[0] = 4
                        if(player == "p1"):
                            return 1
                        else:
                            return -1
                    else:#nao pode haver empate
                        state[0] = 5
                        if(player == "p1"):
                            return -1
                        else:
                            return 1

    def startGame(self):
        status = 1 #1=in progress; 2=one player blocked;3=two players blocked;4/5=p1 won/p2 won
        pieces = [(x,y) for x in range(7) for y in range(x,7)]
        random.shuffle(pieces)
        p1_hand = self.buy([],7,pieces)
        p2_hand = self.buy([],7,pieces)
        field = []
        l_end = -1
        r_end = -1
        state = [status, p1_hand, p2_hand, field, l_end, r_end]
        return state

    def playGame(self,state,action,player):
        status = state[0]
        if(status >= 3): #jogo acabou
            return state
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
            
        field = state[3]
        l_end = state[4]
        r_end = state[5]

        if(action[0] is None):#foi bloqueado
            if(status == 2):
                state[0] = 3
            elif(status == 1):
                state[0] = 2
            return state
        
        if(status == 2): # o oponente foi bloqueado, mas eu tenho peça
            state[0] = 1
        p_index = action[2]  
        if(l_end==r_end==-1):
            orientation = "left"
        else:
            orientation = action[1]
        p = hand[p_index]
        field.append(p)
        hand.remove(p)
        if (l_end==-1 and r_end==-1):
            l_end, r_end = p
        elif (orientation == "right"):#ori e o lado desejado a manter na ponta
            r_end=action[0]
        elif (orientation == "left"):
            l_end=action[0]
                
        if(player =="p1"):
            state[1] = hand
        else:
            state[2] = hand
            
        if(len(hand) ==0): #zerou a mao, acabou o jogo
            state[0] = 3
        state[3] = field
        state[4] = l_end
        state[5] = r_end
        return state



In [3]:

class Features:
    
    def __init__(self,beta,featureWeights):
        self.data = []
        self.beta = beta
        self.featureWeights = featureWeights
        self.numFeatures = featureWeights.size
        
    def getFeaturesWeights(self):
        return self.featureWeights
    
    def featureValues(self,state,action,player):
        result =  np.zeros(self.numFeatures)
        l_end = state[4]
        r_end = state[5]
        if(player == "p1"):
            hand = state[1]
            handOp = state[2]
        elif(player == "p2"):
            hand = state[2]
            handOp = state[1]
        if(action[0]!=None):#quando nao for bloqueado    
            pos = action[2]
            if(l_end==r_end==-1):
                pos = action[2]
                l_end = action[0]
                r_end = action[1]
            elif(action[1]=="left"):
            #substitui uma das pontas pela nova ação
                l_end = action[0]
            else:
                r_end = action[0]
            #print "pos: "+str(pos)
            value =hand.pop(pos)
        """
        Características a partir de informações imperfeitas.
            Somente do que é visível a um jogador comum:
                Campo, peças de suas mãos, quantidade de peças do oponente
        """
        #valida caracteristicas de acordo com o estado submetido e as retorna
        
        result[0] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==1)
        result[1] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==2)
        result[2] = self.NumDouble(hand,action,l_end,r_end,lambda x: x>=3)
        #impossibilita qualquer peça de ser jogada
        result[3] = self.blocks(hand,action,l_end,r_end)
        
        result[4] = self.NumActions(hand,l_end,r_end,lambda x: x==1)
        result[5] = self.NumActions(hand,l_end,r_end,lambda x: x==2)
        result[6] = self.NumActions(hand,l_end,r_end,lambda x: x>=3)
       
        for i in range(7,14):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = self.hasDouble(i-7,hand)
                
        """
        Características a partir de informações perfeitas.
        Sabe-se da mão do oponente, de toda e qualquer informação necessária
        para se tomar uma decisão
        """
        result[14] = self.blocks(handOp,action,l_end,r_end)
        result[15] = self.NumActions(handOp,l_end,r_end,lambda x: x==1)
        result[16] = self.NumActions(handOp,l_end,r_end,lambda x: x==2)
        result[17] = self.NumActions(handOp,l_end,r_end,lambda x: x>=3)
        for i in range(18,25):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = self.shortPiece(i-18,handOp)
        
        if(self.numFeatures>25):
            result[25] = self.hasPieceValue(hand,lambda x: x>=9)
            result[26] = self.hasPieceValue(hand,lambda x: x<=3)
            result[27] = self.hasHandValue(hand,lambda x: x<=4)
            result[28] = self.hasHandValue(hand,lambda x: x>=12 and x<=19)
            result[29] = self.hasHandValue(hand,lambda x: x>=20)
            
            result[30] = self.hasPieceValue(handOp,lambda x: x>=9)
            result[31] = self.hasPieceValue(handOp,lambda x: x<=3)
            result[32] = self.hasHandValue(handOp,lambda x: x<=4)
            result[33] = self.hasHandValue(handOp,lambda x: x>=12 and x<=19)
            result[34] = self.hasHandValue(handOp,lambda x: x>=20)
        
        
        if(action[0]!=None):
            hand.insert(pos,value)
        
        return result
    
    
    def NumDouble(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(piece[0] == piece[1]):
                count +=1
        if(exp(count)):
            return True
        return False
    
    def hasDouble(self,number,hand):
        for piece in hand:
            if((piece[0] == piece[1]) and (piece[0] == number)):
                return True
        return False
    
    def hasPieceValue(self,hand,exp):
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                return True
        return False
    
    def hasHandValue(self,hand,exp):
        value = 0
        for piece in hand:
            value += piece[0] + piece[1]
        if(exp(value)):
            return True
        else:
            return False
    
    def blocks(self,hand,action,l_end,r_end):
        if(action[0] is None):
            return True
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                #se encontra alguma peça valida na mao
                return False # então ainda não estou bloqueado
  
        return True #nao encontrou
        
    def NumActions(self,hand,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                count +=1
        if(exp(count)):#se tem mais que o numero de peças designado
            return True
        return False #se tem exatamente aquele numero
    
    def shortPiece(self,number,handOp):
        for piece in handOp:
            if(piece[0]==number or piece[1]==number):
                return False
        return True #op n tem nenhuma peça dese tipo
        
          
    def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
        if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
            return True
        return False
    



In [5]:
#dado o jogador, sua acao, o estado e suas caracteristicas, retorne o valor dessa acao
#e necessario diferencia os jogadores pois ele precisa examinar 2 hands 
#o act n especifica de que hand e
def qValue(state,act,playerFeatures,player):
    featureVals = playerFeatures.featureValues(state,act,player)
    val = np.dot (playerFeatures.getFeaturesWeights(), featureVals)    # value before action
    return val
        
def eGreedyPicker(actions,state,playerFeatures,player):
    rand = np.random.random_sample()
    bestValue = 0 #tomar cuidado aqui
    cond = False
    if(rand > e):
        for act in actions:  
            value = qValue(state,act,playerFeatures,player)
            if(not cond):
                bestValue = value
                choosenAct = act
                cond = True
            if(value> bestValue):
                bestValue = value
                choosenAct = act
    else:
        choosenAct = random.choice(actions)
  
    return choosenAct

    
def policyAct(state,player,playerFeatures):
    actions = dominoes.possibleActions(state,player)
    #escolhe a partir dos valores na q-value com tecnica e-greedy
    if(actions[0] is None):
        return actions
    else:
        choosenAct = eGreedyPicker(actions,state,playerFeatures,player)
    
    return choosenAct

def step(state,player,act):
    state = dominoes.playGame(state,act,player)                  # do selected action
    #print player + " played action : "+str(act)
    #print "now state is " + str(state)
    return state

def alpha(vector):#calcua taxa de aprendizado de acordo com quantidade de features atuais
    unique, counts = np.unique(vector, return_counts=True)
    a = 0.1/(counts[1]) #constante por numero total de 1
    return a
def evaluate(state,player,val,act,playerFeatures,featureWeights,featuresVector):
    #A recompensa é dada após a ação dos 2 players
    #não é recompensa pela ação de 1 player, mas dos dois
    r = dominoes.reward(state,player)   
    next_action = policyAct(state,player,playerFeatures)                 
    new_value = qValue(state,next_action,playerFeatures,player)
    target = r+ 0.9 * new_value                     # gamma = 0.9
    delta = target - val                            # prediction error
    featureWeights += alpha(featuresVector) * delta *featuresVector
    elements = [state,new_value,next_action,featureWeights]
    return elements
    


In [6]:
dominoes = Domino()
e = 0.03  # epsilon-greedy proportion
featureWeights = np.zeros(35)
features = Features(1,featureWeights)
r = 0
duration = 0
#self-play
firstplay = 0
for i in range(0,10000):
    state = dominoes.startGame()
    #firstplay = random.randint(0,1)#2 jogadores compartilhando a msm rede neural,mas so 1 avaliando

    #print "Game "+str(i)
    #print "Initial state:" +str(state)
    #actionss1 = dominoes.possibleActions(state,"p1")
    actP1 = policyAct(state,"p1",features)         # take action by a e-greedy policy
    val1 = qValue(state,actP1,features,"p1")
    flag = 0
    while(dominoes.termination(state) == False):
        if(firstplay==0):#salva vetor de carac antes de mudar
            featuresVector1 = features.featureValues(state,actP1,"p1")
        else:
            featuresVector2 = features.featureValues(state,actP2,"p2")  
        state = step(state,"p1",actP1)
        ##print "valid features are: "+str(featuresVector)
        
        if(flag==0):
            actP2 = policyAct(state,"p2",features)
            flag = 1
        val2 = qValue(state,actP2,features,"p2")
        state = step(state,"p2",actP2)

        #pega recompensas, atualiza as características e retorna novo
        #estado, valor e proxima acao
        if(firstplay==0):#desse modo a msm rede neural e avaliada as vezes como p1 ou p2
            state,val1,actP1,featureWeights =evaluate(state,"p1",val1,actP1,features,featureWeights,featuresVector1)
            actP2 = policyAct(state,"p2",features)
        else:
            state,val2,actP2,featureWeights =evaluate(state,"p2",val2,actP2,features,featureWeights,featuresVector2)
            actP1 = policyAct(state,"p1",features)
    #print "Ending state: "+str(state)
    #print "Resulting weights: "+str(featureWeights)

print "Resulting weights: "+str(featureWeights)
np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet5', featureWeights)

Resulting weights: [-0.17547866 -0.15514322 -0.08801508  0.9467075   0.99171707  0.98396033
  0.94567308  0.25196739 -0.00861509 -0.19492712 -0.20133019 -0.16652455
 -0.09293869 -0.22839716 -0.05607874  0.02569663 -0.09296799 -0.17399491
  0.51315692  0.02823395  0.14092833  0.26141676  0.20065675  0.20482948
  0.26932737  0.05453038 -0.09369467  1.59841699 -0.55086085 -0.88405523
  0.39709503 -0.16369437 -1.08321733  0.61540473  1.13285755]


In [115]:
dominoes = Domino()
e = 0.0001  # aqui n ha aprendizagem
featureWeightsP1 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet4.npy')
featureWeightsP2 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet5.npy')
featuresP1 = Features(1,featureWeightsP1)
featuresP2 = Features(1,featureWeightsP2)

r = 0
duration = 0

P1Wins = 0
firstplay = 1
for i in range(0,10000):
    state = dominoes.startGame()
    #firstplay = random.randint(0,1)
    while(dominoes.termination(state) == False):
        if(firstplay==0):
            actP1 = policyAct(state,"p1",featuresP1)         # take action by a e-greedy policy
            state = step(state,"p1",actP1)
            actP2 = policyAct(state,"p2",featuresP2)
            state = step(state,"p2",actP2)
        else:
            actP2 = policyAct(state,"p2",featuresP2)         # take action by a e-greedy policy
            state = step(state,"p2",actP2)
            actP1 = policyAct(state,"p1",featuresP1)
            state = step(state,"p1",actP1)
        r = dominoes.reward(state,"p1")
    if(state[0]==4):
        P1Wins+=1


print "Resulting weights: "+str(featureWeightsP1)
#np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet3', featureWeightsP1)
print "Player1 winnings: "+str(P1Wins)

Resulting weights: [-0.09192529 -0.20659979 -0.18093689  1.51690381  1.39831485  1.3739857
  1.47406724  0.39218379  0.23245208 -0.12492196 -0.3718462  -0.41579741
 -0.3056125  -0.45784327 -0.13942727 -0.40643513 -0.63778834 -0.7466368
  0.22032436  0.21868294  0.06141763  0.12064992 -0.04507749 -0.01658206
  0.0762826  -0.27093872 -0.10939788  1.18334184 -0.36133493 -0.82256886
  0.30248491 -0.32532836 -0.90514442  0.6470053   1.17833832]
Player1 winnings: 3765


In [48]:
a = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet.npy')
a

array([-0.18361847, -0.14694126, -0.16895149,  1.36126229,  1.14012367,
        1.18473559,  1.15539816,  0.42505366, -0.03573196, -0.1763768 ,
       -0.16318463, -0.32264377, -0.34663992, -0.39625977, -0.17697972,
       -0.27019893, -0.26841684, -0.31552921,  0.48038823,  0.17342522,
        0.05557403, -0.05822544, -0.05143658, -0.16817866, -0.17066361])

In [54]:
b = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet2.npy')
b

array([ -2.05998979e-02,  -2.62488506e-02,   5.11928778e-04,
         2.93191600e-02,   7.90449338e-03,   9.08829636e-03,
         2.51276034e-03,   8.38678809e-02,  -1.59175326e-02,
        -1.25717913e-02,  -1.94997232e-02,  -3.73431427e-02,
        -4.04316603e-02,  -2.96658434e-02,   1.05623371e-02,
        -4.68152157e-02,  -1.08597596e-04,  -7.05953077e-04,
         1.11955643e-01,  -7.86040757e-02,  -2.07987917e-02,
        -4.24234825e-02,  -4.65862257e-02,   4.39108207e-02,
        -1.55914805e-02,  -7.11837625e-02,   1.15081381e-01,
         2.06545406e-01,  -3.54756672e-02,  -5.90616192e-02,
         5.09523125e-02,  -5.28248819e-02,  -7.41968084e-02,
         8.74292495e-02,   6.44455089e-02])

In [79]:
c = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet3.npy')
c

array([-0.152977  , -0.13401844, -0.21405536,  1.47267357,  1.27856345,
        1.26356787,  1.2484516 ,  0.41680209, -0.07704588, -0.10598451,
       -0.15727729, -0.3534382 , -0.40956067, -0.44606388, -0.1514356 ,
       -0.23440379, -0.28698809, -0.36992121,  0.48223835,  0.15121347,
        0.10701091,  0.17787953, -0.05757216, -0.03968698, -0.07373018])

In [16]:
featureWeightsP1 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet4.npy')
features1 = Features(1,featureWeightsP1)

In [8]:
dominoes = Domino()
e = 0.03  # epsilon-greedy proportion
state = dominoes.startGame()
#print "Initial state:" +str(state)
actionss1 = dominoes.possibleActions(state,"p1")
actP1 = policyAct(state,"p1",features)         # take action by a e-greedy policy
val1 = qValue(state,actP1,features,"p1")
#for i in range(0,1): 
while(dominoes.termination(state) == False):
        featuresVector = features.featureValues(state,actP1,"p1")
        state = step(state,"p1",actP1)
        print "State after p1 play: "+str(state[4]) + ","+str(state[5])
        print "my hand: "+str(state[2])
        actionss2 = dominoes.possibleActions(state,"p2")
        if(actionss2[0] is None):
            actP2 = policyAct(state,"p2",features)
            state = step(state,"p2",actP2)
        else:
            actionPos=input("Choose Action: "+str(actionss2))
            state = step(state,"p2",actionss2[actionPos])
        state,val1,actP1,featureWeights =evaluate(state,"p1",val1,actP1,features,featureWeights,featuresVector)
        print "State after both plays: "+str(state[0])

State after p1 play: 3,3
my hand: [(5, 6), (5, 5), (1, 3), (2, 2), (2, 4), (4, 6), (0, 0)]
Choose Action: [(1, 'left', 2)]0
State after both plays: 1
State after p1 play: 1,3
my hand: [(5, 6), (5, 5), (2, 2), (2, 4), (4, 6), (0, 0)]
State after both plays: 2
State after p1 play: 1,0
my hand: [(5, 6), (5, 5), (2, 2), (2, 4), (4, 6), (0, 0)]
Choose Action: [(0, 'right', 5)]0
State after both plays: 1
State after p1 play: 1,0
my hand: [(5, 6), (5, 5), (2, 2), (2, 4), (4, 6)]
State after both plays: 4
